In [35]:
from mapper import mapProperty, mpNoRank, mpNoRankWType, mpRank, mpRankWType
import csv
import pandas as pd
import numpy
from sklearn.utils import resample
import csv
import pandas as pd
from wikimedia import searchEntity, searchObjWProperty, searchProperty
from dateutil.parser import parse
import operator
from indexer import connect_elasticsearch, search
import logging
import json
from statistics import mean 

In [101]:
def lDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def check_correctness(data, mapping):
    result_map = {}
    for item in list(data['nama kolom']):
        if(mapping[item] == data['property'][data.loc[data['nama kolom'] == item].index[0]]):
            result_map[item] = 1
        elif(mapping[item] == '' and pd.isnull(data['property'][data.loc[data['nama kolom'] == item].index[0]])):
            result_map[item] = 1
        else:
            result_map[item] = 0
    return result_map

def countCorrect(map_result):
    return sum(map_result.values())

def search_best_thresh(data, dataset, datatype):
    res_map = {}
    for y in range(4):
        for x in range(10):
            _, mapLabel = mpNoRankWType(dataset, datatype, (x / 10.0) + y)
            cor_map = check_correctness(data, mapLabel)
            res_map['{}.{}'.format(y,x)] = countCorrect(cor_map)
    return res_map

def precision(nums_correct, total):
    return nums_correct / total

def avePrecision(relevance, res):
    precision_score = []
    nums_correct = 0
    total = 0
    for elem in res:
        total = total + 1
        if(elem['_source']['label'].lower() == relevance.lower()):
            nums_correct = nums_correct + 1
            precision_score.append(precision(nums_correct, total))
        elif(relevance in elem['_source']['alias']):
            nums_correct = nums_correct + 1
            precision_score.append(precision(nums_correct, total))
            
    if(len(precision_score) > 0):
        return mean(precision_score)
    else:
        return 0

In [32]:
def build_query(elem, dt='', fuzziness=False):
    search_object = ''
    if(dt != ''):
        if(fuzziness):
            search_object = {
                "query": {
                    "bool": {
                        "should": {
                            "match": {
                                "alias": {
                                    "query": elem,
                                    "fuzziness": "AUTO"
                                }
                            }
                        }
                    }
                }
            }
        else:
            search_object = {
                "query": {
                    "bool": {
                        "should": {
                            "match": {
                                "alias": {
                                    "query": elem
                                }
                            }
                        }
                    }
                }
            }
    else:
        if(fuzziness):
            search_object = {
                "query": {
                    "bool": {
                        "filter": {
                            "term": {"data type": type}
                        },
                        "should": {
                            "match": {
                                "alias": {
                                    "query": elem,
                                    "fuzziness": "AUTO"
                                }
                            }
                        }
                    }
                }
            }
        else:
            search_object = {
                "query": {
                    "bool": {
                        "filter": {
                            "term": {"data type": type}
                        },
                        "should": {
                            "match": {
                                "alias": {
                                    "query": elem
                                }
                            }
                        }
                    }
                }
            }
    return search_object

def mp(header_list, elem, type_list, use_type=True, fuzziness=True):
    logging.basicConfig(level=logging.ERROR)
    property_index = 'wd_property'
    entity_index = 'wd_entity'
    property_doc_type = 'members'
    es = connect_elasticsearch()

    search_object = ''
    elem = elem.replace('_', ' ')
    if(use_type):
        search_object = build_query(elem, type_list[header_list.index(elem)], fuzziness)
    else:
        search_object = build_query(elem, '', fuzziness)

    res = search(es, property_index, json.dumps(search_object))['hits']['hits']
    return res

def mp2(header_list, type_list, thresh=0):
    logging.basicConfig(level=logging.ERROR)
    es = connect_elasticsearch()

    result = {}
    resultLabel = {}
    for elem in header_list:
        elem = elem.replace('_', ' ')
        search_object = build_query(elem, type_list[header_list.index(elem)])
        
        res = search(es, property_index, json.dumps(search_object))['hits']['hits']
        if(len(res) > 0 and res[0]['_score'] > thresh):
            result[elem] = res[0]['_source']['id']
            resultLabel[elem] = res[0]['_source']['label']
        else:
            result[elem] = ''
            resultLabel[elem] = ''
    return result, resultLabel

In [70]:
df_downsampled = pd.read_csv("data/testing/prop_dataset_sampling.csv", encoding='latin-1')
df_downsampled = df_downsampled.dropna()
dataset = list(df_downsampled['nama kolom'])
datatype = list(df_downsampled['tipe data'])
df_downsampled

,nama kolom,tipe data,property,isnull
66,alamat,String,alamat,False
67,alamat jalan,String,alamat,False
68,alamat rs,String,alamat,False
69,alamat sekolah,String,alamat,False
70,bentuk pendidikan,String,adalah,False
71,bentuk reklame,WikibaseItem,adalah,False
72,desa kelurahan,WikibaseItem,terletak di wilayah administrasi,False
73,direktur rs,WikibaseItem,manajer/direktur,False
74,email,Url,surel,False
75,fax,String,nomor faks,False


In [98]:
relevance = list(df_downsampled['property'])

In [102]:
apList = []
for elem in dataset:
    res = mp(dataset, elem, datatype, True)
    apList.append(avePrecision(relevance[dataset.index(elem)], res))

mean(apList)

Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES 

0.37737493987493986

In [103]:
apList

[1.0,
 1.0,
 1.0,
 1.0,
 0,
 0,
 0.5,
 1.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0.125,
 0.16666666666666666,
 0.3333333333333333,
 1.0,
 1.0,
 0,
 0.3333333333333333,
 0.14285714285714285,
 1.0,
 1.0,
 0,
 0,
 0.3333333333333333,
 1.0,
 0,
 0.1111111111111111,
 0,
 0,
 0,
 0,
 0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 0,
 1.0,
 0,
 0.1111111111111111,
 1.0,
 0,
 0,
 0.5,
 0,
 0,
 0,
 0.125,
 1.0,
 1.0,
 0,
 0,
 0,
 0,
 0.125]

In [78]:
search_best_thresh(df_downsampled, dataset, datatype)

Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service
Connected to ES Service


{'0.0': 24,
 '0.1': 24,
 '0.2': 22,
 '0.3': 22,
 '0.4': 21,
 '0.5': 20,
 '0.6': 19,
 '0.7': 19,
 '0.8': 18,
 '0.9': 9,
 '1.0': 2,
 '1.1': 2,
 '1.2': 2,
 '1.3': 2,
 '1.4': 1,
 '1.5': 0,
 '1.6': 0,
 '1.7': 0,
 '1.8': 0,
 '1.9': 0,
 '2.0': 0,
 '2.1': 0,
 '2.2': 0,
 '2.3': 0,
 '2.4': 0,
 '2.5': 0,
 '2.6': 0,
 '2.7': 0,
 '2.8': 0,
 '2.9': 0,
 '3.0': 0,
 '3.1': 0,
 '3.2': 0,
 '3.3': 0,
 '3.4': 0,
 '3.5': 0,
 '3.6': 0,
 '3.7': 0,
 '3.8': 0,
 '3.9': 0}

In [81]:
_, mapLabel = mpNoRankWType(dataset, datatype, 0.1)
cor_map = check_correctness(df_downsampled, mapLabel)
list(mapLabel.values())

Connected to ES Service


['alamat',
 'alamat',
 'alamat',
 'alamat',
 '',
 'benua',
 'terletak di wilayah administrasi',
 'manajer/direktur',
 '',
 'kode bandar udara FAA',
 'jenis kelamin',
 'jenis kelamin',
 'jenis kelamin',
 'jenis kelamin',
 'jenis kelamin',
 'jenis kelamin',
 'lokasi pada bentang alam',
 'Jumlah',
 'Jumlah',
 'Jumlah',
 'Jumlah',
 'Jumlah',
 'Jumlah',
 'kategori mengandung',
 'terletak di wilayah administrasi',
 'kepala negara',
 'kode pos',
 'kode',
 '',
 'kota kembar',
 'kota kembar',
 'terletak di wilayah administrasi',
 'lokasi sitogenetik',
 'koordinat tempat',
 'lokasi sitogenetik',
 'lokasi anatomis',
 'lokasi anatomis',
 'lokasi anatomis',
 'lokasi sitogenetik',
 'luas wilayah',
 'luas wilayah',
 'luas wilayah',
 'luas wilayah',
 'luas wilayah',
 'luas wilayah',
 'masa kerja (mulai)',
 'nama pemberian',
 'nama pemberian',
 'nomor telepon',
 '',
 'nomor edisi',
 'nomor telepon',
 'penyelenggara',
 'kota kembar',
 'tanggal terbit',
 'tanggal lahir',
 'tanggal lahir',
 'tanggal lahir